In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow_tts.processor.ljspeech import LJSpeechProcessor
from tensorflow_tts.processor.ljspeech import symbols, _symbol_to_id
from tensorflow_tts.configs import FastSpeech2Config
from tensorflow_tts.models import TFFastSpeech2

from tensorflow_tts.models import TFMelGANGenerator
from tensorflow_tts.models import TFPQMF
from tensorflow_tts.configs import MultiBandMelGANGeneratorConfig, MelGANGeneratorConfig

import noisereduce as nr

import IPython.display as ipd

/home/qblocks/anaconda3/envs/fastspeech/lib/python3.8/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
!which python

/home/qblocks/anaconda3/envs/fastspeech/bin/python


### Utility Function - 

In [3]:
from  abbreviation import abb_dict

def expand_abbreviationsCustom(text):
    text = text.lower()
    text = text.split()
    new_text =[]
    for word in text:
        word = word
#         print(word)
        if word in abb_dict.keys():
            new_text.append(abb_dict[word])
#             print("abb_expan - ", word , abb_dict[word])
        else:
            new_text.append(word)
    
    return " ".join(new_text)
        

    
def sentenceSplitTTS(input_text, model):
    '''
        Splitting the sentences on full stop to handle large sentences. 
    
    '''
    textlist = input_text.split(". ")
    audiolist =[]
    for text in textlist:
        mels, audios = do_synthesis(text, model, melgan, "FASTSPEECH", "MELGAN")
        audiolist.append(audios)
    audios = np.concatenate(audiolist, axis=0)
    return audios
     
    

def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
    input_text = expand_abbreviationsCustom(input_text)
    input_text = " " + input_text  # To handle single word as input
#     print("input Text after expansion - ",input_text)
    input_ids = processor.text_to_sequence(input_text)
    input_ids = np.concatenate([input_ids, [len(symbols) - 1]], -1)

  # text2mel part
    if text2mel_name == "TACOTRON":
                    _, mel_after, stop_token_prediction, alignment_history = text2mel_model.inference(
                                                                    tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
                                                                    tf.convert_to_tensor([len(input_ids)], tf.int32),
                                                                    tf.convert_to_tensor([0], dtype=tf.int32)
                                                                    )
            

#                 mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
#                                                                 tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
#                                                                 attention_mask=tf.math.not_equal(tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0), 0),
#                                                                 speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
#                                                                 speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 )            
            
    elif text2mel_name == "FASTSPEECH":
        
        
                mel_before, mel_after, duration_outputs = fastspeech.inference(
                                                                input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
                                                                speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
                                                                speed_ratios=tf.convert_to_tensor(soundReplaySpeed, dtype=tf.float32),
                                                                )
#                  mel_before, mel_after, duration_outputs = text2mel_model.inference(
#                                                                  input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
#                                                                   attention_mask=tf.math.not_equal(tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0), 0),

#                                                                  speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
#                                                                  speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                  )

    elif text2mel_name == "FASTSPEECH2":
        
               
                mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
                                                                input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
                                                                speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
                                                                speed_ratios=tf.convert_to_tensor(soundReplaySpeed, dtype=tf.float32),
                                                                f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
                                                                energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
                                                                )
        
        
#                 mel_before, mel_after, duration_outputs, _, _ = text2mel_model.inference(
#                                                                 tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
#                                                                 attention_mask=tf.math.not_equal(tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0), 0),
#                                                                 speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
#                                                                 speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
#                                                                 )
    else:
        raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

    
    mel_outputs = mel_after
    
    # vocoder part
    if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
        audio = vocoder_model(mel_outputs)[0, :, 0]
    elif vocoder_name == "MB-MELGAN":
        audio = vocoder_model(mel_outputs)
        audio = pqmf.synthesis(audio)[0, :, 0]
    else:
        raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

    if text2mel_name == "TACOTRON":
        return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
    else:
        return mel_outputs.numpy(), audio.numpy()
    
    

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
  plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
  plt.show()
  plt.close()

In [4]:
baseDir = "/home/qblocks/TensorFlowTTS/examples/"
preTrainedModelPath = "../pretrained_model/"

rate= 16000

soundReplaySpeed = [1.15]

# Load and Inference

# Load Models -

## Load Fastspeech2

In [5]:
!ls /home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-*

/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-100000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-105000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-110000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-115000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-120000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-125000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-130000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-135000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-140000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-145000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-150000.h5
/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/model-30000.h5
/home/qblocks/TTS_models/turk

In [6]:
########### Loading config file for fastspeech2 model ..

with open(baseDir + "fastspeech2/conf/fastspeech2.v2.yaml") as f:
    config = yaml.load(f, Loader=yaml.Loader)
    
config = FastSpeech2Config(**config["fastspeech2_params"])
processor = LJSpeechProcessor(None, "english_cleaners")


fastspeech2 = TFFastSpeech2(config=config, name="fastspeech2")

########### load the trained model ...

# fastspeech2.load_weights("exp/train.fs2.v1/checkpoints/model-20000.h5")
fastspeech2ModelPath = "/home/qblocks/TTS_models/turkish/male/fs2_tr_male.v1/checkpoints/"

fastspeech2._build()
# fastspeech2.load_weights( fastspeech2ModelPath + "model-90000.h5")
fastspeech2.load_weights( fastspeech2ModelPath + "model-150000.h5")


## Fastspeech

In [7]:
!ls /home/qblocks/TTS_models/turkish/male/fs_tr_male.v1/checkpoints/model-*

/home/qblocks/TTS_models/turkish/male/fs_tr_male.v1/checkpoints/model-55000.h5
/home/qblocks/TTS_models/turkish/male/fs_tr_male.v1/checkpoints/model-55528.h5


In [8]:
from tensorflow_tts.configs import FastSpeechConfig
from tensorflow_tts.models import TFFastSpeech


########### Loading config file for fastspeech2 model ..

with open('../examples/fastspeech/conf/fastspeech.v3.yaml') as f:
    config = yaml.load(f, Loader=yaml.Loader)
    
config = FastSpeechConfig(**config["fastspeech_params"])
processor = LJSpeechProcessor(None, "english_cleaners")


fastspeech = TFFastSpeech(config=config, name="fastspeech")


########### load the trained model ...


fastspeech._build()

fastspeechModelPath = "/home/qblocks/TTS_models/turkish/male/fs_tr_male.v1/checkpoints/"
fastspeech.load_weights(fastspeechModelPath + "model-55000.h5")


## MelGAN 

In [9]:
!ls /home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-25*

/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2500000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2510000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2520000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2530000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2540000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2550000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2560000.h5
/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/generator-2562842.h5


In [10]:
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import TFAutoModel

trainMelGanPath = "/home/qblocks/TTS_models/turkish/male/malgan_tr_male.v1/checkpoints/" 


# trainMelGanFile = trainMelGanPath + "generator-1530000.h5"

trainMelGanFile = trainMelGanPath + "generator-2560000.h5"




melConfigPath = "/home/qblocks/TensorFlowTTS/examples/melgan/conf/"


config = AutoConfig.from_pretrained(melConfigPath + "melgan.v1.yaml")

melgan = TFAutoModel.from_pretrained(config= config,
                                     pretrained_path= trainMelGanFile,
                                     is_build=True,   # don't build model if you want to save it to pb. (TF related bug)
                                     name="melgan_generator"
                                    )

## Input text - (test texts ) 

In [11]:

input_text = "Modern siyaset dünyasında, dünya ticaretini dünya sağlık hizmetlerine daha küçük kuruluşlara \
            odaklayan ve küçük ülkelerden Rusya ve Amerika Birleşik Devletleri gibi dünyanın güç ülkelerine kadar \
            küçük ülkelerden oluşabilen uluslararası kuruluşlar vardır. Bu uluslararası kuruluşlar, küresel \
            politika ölçeğinde çok büyük bir rol oynamaktadır ve etkilerini anlamaya yardımcı olmak için, \
            küresel ölçekte genel etkilerini açıklamaya yardımcı olmak için farklı teoriler oluşturulmuştur.\
            Yeni Gerçekçilik, Yeni-kurumsalcılık, Neo-liberalizm ve yapılandırmacılık gibi ana teorilerden \
            dördü ve bu teorilerin her birinin aralarında benzerlikleri ve farklılıkları vardır, bazılarına göre, \
            uluslararası örgütleri küresel aşamada daha iyi tanımlar."

# input_text = "Siyasetin ne olduğu sorulduğunda, çoğu kişi, halkın seçildikten sonra tutamayacak sözler \
#                 veren, düzgün bir şekilde bir araya getirilen politikacıların söylediği iyi yapılandırılmış \
#                 yalanlar nedeniyle insanları oylarına mahkum etmenin başka bir yolu olduğunu söyler."

# english = "If asked what politics is, most would usually say that it is another way of conning people for their votes\
#             due to well-constructed lies told by decently put together politicians making promises toward the public \
#             that cannot be kept once elected."

# input_text = "Bu sefer corona virüsü çok sayıda ülkeye yayıldı ve hükümetler çok çaba sarf etseler de, vaka sayısındaki artışın nasıl \
#                 azaltılabileceği ve gezegen bu krizden ne zaman genel bir rahatlama göreceği belli değil. ."

# english =  "This time the corona virus has spread to large number of countries and even though \
#                 the governments are making lot of effort, it is not clear how the growth in the number of \
#                 cases can be reduced and when will the planet see an overall relief from this crisis. "

# input_text = "Bay Charlie, Bayan Khurana, Dr Sameera ve Col Donald Trump başarılı ve başarı kolay gelmiyor. \
#                Çok stresli geceler, tıslama, işeme, özledim, yaygara, giyinme gerektirir. Ders en kısa sürede başlayacak"

# english = "Mr Charlie , Mrs Khurana , Dr Sameera and Col Donald Trump  are successful and success does not \
#         come with ease. It takes lots of stressful nights, hiss, piss, miss, fuss,dress. Class is going to start ASAP"


input_text = "Tarihte incelenen çeşitli düşünürlerden birçoğu siyaset dünyasına katkıda bulunmuş \
                ve günümüz hükümetinin yolunu açmıştır."

### FastSpeech2 + MELGAN

In [12]:
rate = 24000

soundReplaySpeed = [1.10]


audios = sentenceSplitTTS(input_text, fastspeech2)
ipd.Audio(audios, rate=rate)

In [18]:
print(input_text)

Tarihte incelenen çeşitli düşünürlerden birçoğu siyaset dünyasına katkıda bulunmuş                 ve günümüz hükümetinin yolunu açmıştır.


In [13]:
reduced_noise = nr.reduce_noise(audio_clip=audios,
                                noise_clip=audios,
                                verbose=False,
                                use_tensorflow=False,
                                prop_decrease=0.85
                               )
ipd.Audio(reduced_noise, rate=rate)

### FastSpeech + MELGAN

In [14]:
rate = 24000

soundReplaySpeed = [1.1]

In [15]:
%%time
audios = sentenceSplitTTS(input_text, fastspeech)
ipd.Audio(audios, rate=rate)


CPU times: user 46.8 ms, sys: 27.7 ms, total: 74.4 ms
Wall time: 63.4 ms


In [16]:
print(input_text)

Tarihte incelenen çeşitli düşünürlerden birçoğu siyaset dünyasına katkıda bulunmuş                 ve günümüz hükümetinin yolunu açmıştır.


In [17]:
reduced_noise = nr.reduce_noise(audio_clip=audios,
                                noise_clip=audios,
                                verbose=False,
                                use_tensorflow=False,
                                prop_decrease=0.85
                               )
ipd.Audio(reduced_noise, rate=rate)